In [15]:
import numpy as np
from scipy.linalg import solve

def jacobi(A, b, x, n):

    D = np.diag(A)
    R = A - np.diagflat(D)
    
    for i in range(n):
        x = (b - np.dot(R,x))/ D
    return x

'''___Main___'''

A = np.array([[4.0, -2.0, 1.0], [1.0, -3.0, 2.0], [-1.0, 2.0, 6.0]])
b = [1.0, 2.0, 5.0]
x = [1.0, 1.0, 1.0]
n = 25

x = jacobi(A, b, x, n)

print (solve(A, b))

[-0.01369863 -0.09589041  0.8630137 ]


In [ ]:
b seismic signal which is constant
x is to be solved, the reflection coefficient
A should be wavelet

In [16]:
np.dot(A,x)

array([1.00000001, 2.00000001, 5.00000001])

In [9]:
# n = len(M)
# rj = [1.0/n for _ in range(n)]
# p = [1.0/n for _ in range(n)]
# n,rj,p
import numpy as np
np.array(M).dot(p),rj

(array([0.33333333, 0.16666667, 0.5       ]),
 [0.3333333333333333, 0.3333333333333333, 0.3333333333333333])

In [12]:
alpha = .8
M = [
    [0, 0, 1],
    [.5, 0, 0],
    [.5, 1, 0],
]
n = len(M)
rj = [1.0/n for _ in range(n)]
p = [1.0/n for _ in range(n)]

for _ in range(100):
    p_old = [x for x in p]
    for i in range(n):
        pi = 0.0
        for j in range(n):
            pi += M[i][j] * p_old[j]
        pi *= alpha
        pi += (1-alpha) * rj[i]
        p[i] = pi
print(p)
rj,np.array(M).dot(p)


[0.3836477987421384, 0.22012578616352202, 0.39622641509433965]


([0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
 array([0.39622642, 0.1918239 , 0.41194969]))

In [ ]:
# % Function Jacobian %
def jacobian(x,r,t,roa,roa1,method='ghosh'):
    lr=len(r)
    lt=len(t)
    par = 0.1;
    r2 = np.array(r);
    roa2=roa3=np.zeros_like(roa1)
    A1=[]
    A2=[]
    for i2 in range( lr):
        r2[i2] = (r[i2]*par)+r[i2];
        roa2=VES1dmodel(r2,t,x,method=method) #VES1dmodYunus(r2,t,x)
        A1.append( ((np.array(roa2-roa1)/(r[i2]*par))*r[i2])/roa);
        r2 = r.copy();

    t2 = np.array(t);
    for i3 in range(lt):
        t2[i3] = (t[i3]*par)+t[i3];
        roa3=VES1dmodel(r,t2,x,method=method)
        A1.append( (np.array(roa3-roa1)/(t[i3]*par))*t[i3]/roa)
        t2 = t.copy();
    return np.array(A1).T
# np.array(A1)
# A1
# jacobian(x,r,t,roa,roa1)
def VES1dInv(m,x,roa,method='ghosh',maxiteration=100):
    kr = 10e-10;
    iteration = 1;
#     maxiteration = 100;
    dfit = 1;
    roa1=np.zeros_like(roa)
    lr=int(1+len(m)/2)
    lt=int(len(m)/2)
    print('Processing.',end='')
    while iteration<maxiteration:
#         print(iteration,iteration%10)
        if iteration%10==1:
            print('.',end='')
        iteration +=1
        r = m[:lr];
        t = m[-lt:];

        roa1=VES1dmodel(r,t,x,method=method)
        e1 = np.array(np.log(roa)-np.log(roa1));
        dd = e1;
        misfit1 = np.dot(e1.T,e1)
    #     if misfit1<kr:
    #         loglog(x,roa,'k.',x,roa1,'k');
    #         axis([1 1000 1 1000])
    #         xlabel('AB/2 (m)');
    #         ylabel('Apparent Resistivity (Ohm-m)');
    #         break

        A= jacobian(x,r,t,roa,roa1,method=method);
    #     U, S, V = svd(A,0);
        U, S, V = np.linalg.svd(A,full_matrices= False)
        ss = len(S);
        say = 1;
        k = 0;

        while say<ss:
        #     say = say+1;
            diagS = S;
            beta = S[say-1]*(dfit**(1/say));
            if beta<10e-5:
                beta = 0.001*say;
            SS=np.zeros((ss,ss))
            for i4 in range(ss):
                SS[i4,i4] = S[i4]/(S[i4]**2+beta);

            dmg = np.linalg.multi_dot([V.T,SS,U.T,dd.T])
            mg = np.exp(np.log(m)+dmg.T);
            r = mg[:lr];
            t = mg[-lt:];
            roa4=VES1dmodel(r,t,x,method=method)


            e2 = np.array(np.log(roa)-np.log(roa4));
            misfit2 = np.dot(e2.T,e2);

            if misfit2>misfit1:
                'Beta control'
                say = say+1;
                k = k+1;
                if k == ss-1:
                    iteration = maxiteration;
                    say = ss+1;

            else:
                say = ss+1;
                m = mg;
                dfit = (misfit1-misfit2)/misfit1;
                iteration = iteration+1;
                a = iteration;
                if dfit<kr:
                    iteration = maxiteration;
                    say = say+1;
    print('\nIterated %d times '%(iteration))
    return mg,roa1
